# Project Echo - Experiment Benchmarking Framework

This notebook provides an interactive interface to the benchmarking framework. It allows you to run various experiments with different model architectures and augmentation strategies, and compare their performance.

## Overview

The benchmarking framework is designed to systematically evaluate different combinations of:
- Model architectures (EfficientNet, MobileNet, ResNet, etc.)
- Audio augmentation strategies
- Image/spectrogram augmentation strategies

Results are collected and visualized to help identify the best performing configurations for bird sound classification.

# 0.1 Install Required Libraries
The following cell is to install required libraries if you are running this notebook remotly, such as on an instance from Vast.ai or google colab.
Ensure you have a clean python 3.9.21 kernal to start.
Details on how to set this up are contained within the readme.

In [ ]:
from ipywidgets import IntSlider
from IPython.display import display
slider = IntSlider()
display(slider)




## 1. Import Required Libraries

In [ ]:
# --- CONFIGURATION FOR DEVCONTAINER ---
import os
import sys

# When running in devcontainer, the workspace is mounted to /workspace
if os.path.exists('/workspace') and os.path.exists('/workspace/config'):
    # Running in devcontainer with proper mount
    base_path = "/workspace"
    print("✓ Running in devcontainer environment")
elif os.path.exists('./config'):
    # Running locally
    base_path = os.getcwd()
    print("✓ Running in local environment")
else:
    # Fallback - try to find config directory
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    
    if os.path.exists(os.path.join(current_dir, 'config')):
        base_path = current_dir
    elif os.path.exists(os.path.join(parent_dir, 'config')):
        base_path = parent_dir
    else:
        print("❌ ERROR: Cannot find config directory")
        print(f"Current directory: {current_dir}")
        print(f"Available files/dirs: {os.listdir(current_dir)}")
        base_path = current_dir

print(f"Using base path: {base_path}")

# Add to Python path
if base_path not in sys.path:
    sys.path.insert(0, base_path)

# Verify required directories exist
required_dirs = ['config', 'utils']
for dir_name in required_dirs:
    dir_path = os.path.join(base_path, dir_name)
    exists = os.path.exists(dir_path)
    print(f"{dir_name} directory: {'✓' if exists else '❌'} {dir_path}")

# --- END CONFIGURATION ---

In [ ]:
# Import necessary libraries
import os
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
from ipywidgets import widgets
from IPython.display import display, HTML, clear_output


# --- CONFIGURATION FOR DEVCONTAINER ---
# When running in devcontainer, use the mounted workspace path
# The devcontainer.json mounts the workspace to /workspace
if os.path.exists('/workspace'):
    # Running in devcontainer
    actual_module_path_inside_container = "/workspace"
    print("Running in devcontainer environment")
else:
    # Fallback for local development
    actual_module_path_inside_container = os.getcwd()
    print("Running in local environment")

print(f"Using module path: {actual_module_path_inside_container}")

if not os.path.isdir(actual_module_path_inside_container):
    print(f"ERROR: The path '{actual_module_path_inside_container}' does NOT exist or is not a directory.")
    print(f"Current CWD: {os.getcwd()}")
    print(f"Available directories: {os.listdir('.')}")
else:
    if actual_module_path_inside_container not in sys.path:
        sys.path.insert(0, actual_module_path_inside_container)
    print(f"Successfully added to sys.path: {actual_module_path_inside_container}")
    
    # Verify the required directories exist
    config_path = os.path.join(actual_module_path_inside_container, 'config')
    utils_path = os.path.join(actual_module_path_inside_container, 'utils')
    print(f"Config directory exists: {os.path.exists(config_path)}")
    print(f"Utils directory exists: {os.path.exists(utils_path)}")

# --- END CONFIGURATION ---

# Import framework components
from config.experiment_configs import EXPERIMENTS

In [ ]:
# Import necessary libraries
import os
import sys
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from ipywidgets import widgets
from IPython.display import display, HTML, clear_output
import re
import importlib

# Add the current directory to path for imports
module_path = os.getcwd() # Gets the current working directory of the notebook
if module_path not in sys.path:
    sys.path.append(module_path)

# Import framework components
from config.experiment_configs import EXPERIMENTS





## 3. Available Experiments

Here you can view and select experiments to run. Each experiment represents a combination of model architecture and augmentation strategies.

## 2. Configuration

Set up the directories and options for benchmarking. 

Ensure to update these in the system_config.py file in the config folder.

The default directories are as follows:

DATA_DIR = "D:\Echo\Audio_data"  # Directory containing audio data

CACHE_DIR = "D:\Echo\Training_cache"  # Directory for caching pipeline results

OUTPUT_DIR = "D:\Echo\results"  # Directory to save experiment results

In [ ]:
# Import directories from system_config
from config.system_config import SC

# Get directory paths from system config
DATA_DIR = SC['AUDIO_DATA_DIRECTORY']
CACHE_DIR = SC['CACHE_DIRECTORY']
OUTPUT_DIR = SC['OUTPUT_DIRECTORY']

print(f"Using directories from system_config:")
print(f"Data Directory: {DATA_DIR}")
print(f"Cache Directory: {CACHE_DIR}")
print(f"Output Directory: {OUTPUT_DIR}")

# Create output directory if it doesn't exist
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)


print("Physical GPUs:", tf.config.list_physical_devices("GPU"))
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("GPU name:", tf.test.gpu_device_name())


# Configure GPU memory if available
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"GPU support enabled: {len(gpus)} GPU(s) found")
else:
    print("No GPU support found, running on CPU")

In [ ]:
# Display available experiments
experiment_data = []
for exp in EXPERIMENTS:
    experiment_data.append({
        "name": exp["name"],
        "model": exp["model"],
        "audio_augmentation": exp["audio_augmentation"],
        "image_augmentation": exp["image_augmentation"],
        "epochs": exp["epochs"],
        "batch_size": exp["batch_size"]
    })

experiments_df = pd.DataFrame(experiment_data)
display(experiments_df)

## 4. Interactive Experiment Selection

Use the widgets below to select experiments and set directories.

In [ ]:
# Create widgets for directory selection
data_dir_widget = widgets.Text(
    value=DATA_DIR,
    description='Data Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

cache_dir_widget = widgets.Text(
    value=CACHE_DIR,
    description='Cache Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

output_dir_widget = widgets.Text(
    value=OUTPUT_DIR,
    description='Output Directory:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

# Group directory widgets
dir_widgets_box = widgets.VBox([data_dir_widget, cache_dir_widget, output_dir_widget])

# Create widget for experiment selection
experiment_options = [(exp["name"], exp["name"]) for exp in EXPERIMENTS]
experiment_widget = widgets.SelectMultiple(
    options=experiment_options,
    description='Select Experiments:',
    disabled=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%', height='200px')
)

# Buttons for actions
run_selected_button = widgets.Button(
    description='Run Selected Experiments',
    button_style='primary',
    tooltip='Run the selected experiments'
)

run_all_button = widgets.Button(
    description='Run All Experiments',
    tooltip='Run all experiments'
)

generate_report_button = widgets.Button(
    description='Generate Report Only',
    button_style='info',
    tooltip='Generate a report from existing results'
)

# Group buttons
buttons_box = widgets.HBox([run_selected_button, run_all_button, generate_report_button])

# Output area for logs
output_area = widgets.Output(layout={'border': '1px solid black', 'width': '90%', 'height': '300px'}) # Adjusted width and added height

# Main container for all control widgets
controls_box = widgets.VBox([
    widgets.HTML("<h3>Directory Configuration:</h3>"), # Optional title
    dir_widgets_box,
    widgets.HTML("<hr><h3>Experiment Selection:</h3>"), # Optional separator and title
    experiment_widget,
    widgets.HTML("<hr><h3>Actions:</h3>"), # Optional separator and title
    buttons_box
])

# Display main controls container and then the output area
display(controls_box)
display(output_area)

## 5. Experiment Runner Functions

These functions handle the execution of experiments and report generation.

In [ ]:
from utils.optimised_engine_pipeline import train_model
import importlib
import config.system_config # Ensure the module is imported

def run_selected_experiments(b):
    """
    Runs experiments based on widget selections.
    Updates configuration in memory instead of writing to file.
    """
    output_area.clear_output(wait=True)
    with output_area:
        print("Starting experiment run...")

        # Get new directory paths from widgets
        new_data_dir = data_dir_widget.value
        new_cache_dir = cache_dir_widget.value
        new_output_dir = output_dir_widget.value

        # --- FIX: Update config directly in memory ---
        # This is safer and more reliable than reloading modules.
        from config.system_config import SC
        SC['AUDIO_DATA_DIRECTORY'] = new_data_dir
        SC['CACHE_DIRECTORY'] = new_cache_dir
        SC['OUTPUT_DIRECTORY'] = new_output_dir
        print("System configuration updated in memory.")
        print(f"  - Data Dir: {SC['AUDIO_DATA_DIRECTORY']}")
        print(f"  - Cache Dir: {SC['CACHE_DIRECTORY']}")

        # Ensure directories exist
        for path in [new_data_dir, new_cache_dir, new_output_dir]:
            if not os.path.exists(path):
                os.makedirs(path, exist_ok=True)
                print(f"Created directory: {path}")

        selected_experiments = list(experiment_widget.value)
        if not selected_experiments:
            print("No experiment selected. Please select at least one experiment.")
            return

        for exp_name in selected_experiments:
            exp_config = next((exp for exp in EXPERIMENTS if exp["name"] == exp_name), None)
            if exp_config is None:
                print(f"Experiment {exp_name} not found in EXPERIMENTS.")
                continue

            print(f"\nRunning experiment: {exp_config['name']}")
            try:
                model, history = train_model(
                    model_name=exp_config['model'],
                    epochs=exp_config.get('epochs'),
                    batch_size=exp_config.get('batch_size')
                )
                print(f"✅ Training completed for experiment: {exp_config['name']}")
                if model:
                    model.summary(print_fn=lambda x: print(x))

            except Exception as e:
                import traceback
                print(f"❌ An error occurred during training for experiment {exp_config['name']}:")
                traceback.print_exc() # Print the full traceback for better debugging

            print("-" * 40)

# Bind the corrected function to the button
run_selected_button.on_click(run_selected_experiments)

print("Experiment runner function has been updated.")


In [ ]:

import tensorflow as tf
from utils.data_pipeline import create_datasets, build_datasets
from config.system_config import SC
from config.model_configs import MODELS

with output_area:
    print("🔬 Starting corrected augmentation diagnostics...")
    try:
        # 1. Get the initial datasets (paths and labels)
        train_ds_init, val_ds_init, test_ds_init, class_names = create_datasets(SC['AUDIO_DATA_DIRECTORY'])
        num_classes = len(class_names)
        print("✅ Initial dataset created.")

        # 2. Get the model's expected input shape
        model_config = MODELS['EfficientNetV2B0'] # Using baseline as an example
        input_shape = model_config['expected_input_shape']
        print(f"✅ Using model input shape: {input_shape}")

        # 3. Build the full data pipeline.
        # This will use the default augmentation strategy defined inside the function.
        train_dataset, _, _ = build_datasets(
            train_ds_init, val_ds_init, test_ds_init, num_classes, input_shape
        )
        print("✅ Full data pipeline built.")

        # 4. Try to pull one batch and see if it fails
        print("⏳ Attempting to get one batch from the augmented dataset...")
        for images, labels in train_dataset.take(1):
            print(f"✅ Successfully retrieved one batch!")
            print(f"   - Image batch shape: {images.shape}")
            print(f"   - Image batch dtype: {images.dtype}")
            print(f"   - Labels batch shape: {labels.shape}")
        print("\n🎉 Diagnostic PASSED. The augmentation pipeline seems to work in isolation.")

    except Exception as e:
        import traceback
        print("\n❌ Diagnostic FAILED. The error was reproduced. See traceback below.")
        print("   This confirms the issue is within the default image augmentation pipeline.")
        traceback.print_exc()

In [ ]:
from utils.optimised_engine_pipeline import train_model

# Update the run_selected_experiments function in your notebook
def run_selected_experiments(b):
    from IPython.display import clear_output
    import importlib
    import config.system_config
    
    output_area.clear_output(wait=True) 
    with output_area:
        print("Starting experiment run...")
        
        # Get new directory paths from widgets
        new_data_dir = data_dir_widget.value
        new_cache_dir = cache_dir_widget.value
        new_output_dir = output_dir_widget.value
        
        # Update the paths directly in the SC dictionary
        from config.system_config import SC
        SC['AUDIO_DATA_DIRECTORY'] = new_data_dir
        SC['CACHE_DIRECTORY'] = new_cache_dir
        SC['OUTPUT_DIRECTORY'] = new_output_dir
        
        # Create directories if they don't exist
        for directory in [new_data_dir, new_cache_dir, new_output_dir]:
            if not os.path.exists(directory):
                os.makedirs(directory, exist_ok=True)
                print(f"Created directory: {directory}")
        
        print(f"Updated directories:")
        print(f"  Data: {new_data_dir}")
        print(f"  Cache: {new_cache_dir}")
        print(f"  Output: {new_output_dir}")

        selected_experiments = list(experiment_widget.value)
        if not selected_experiments:
            print("No experiment selected. Please select at least one experiment.")
            return
        
        for exp_name in selected_experiments:
            exp_config = next((exp for exp in EXPERIMENTS if exp["name"] == exp_name), None)
            if exp_config is None:
                print(f"Experiment {exp_name} not found in EXPERIMENTS.")
                continue

            print(f"Running experiment: {exp_config['name']}")
            
            try:
                # Temporarily disable image augmentation to test if that's the issue
                model, history = train_model(
                    model_name=exp_config['model'],
                    epochs=exp_config.get('epochs', 10),
                    batch_size=exp_config.get('batch_size', 16)
                )
                print(f"Training completed for experiment: {exp_config['name']}")
                if model:
                    model.summary(print_fn=lambda x: print(x))
                    
            except Exception as e:
                print(f"❌ Error running experiment {exp_config['name']}: {str(e)}")
                print("This might be due to image augmentation configuration issues.")
                print("Trying with reduced batch size...")
                
                try:
                    # Try again with smaller batch size
                    model, history = train_model(
                        model_name=exp_config['model'],
                        epochs=exp_config.get('epochs', 10),
                        batch_size=8  # Smaller batch size
                    )
                    print(f"✅ Training completed with reduced batch size for: {exp_config['name']}")
                except Exception as e2:
                    print(f"❌ Still failed with reduced batch size: {str(e2)}")
                    
            print("-" * 40)

# Re-bind the button
run_selected_button.on_click(run_selected_experiments)

In [ ]:
from utils.optimised_engine_pipeline import train_model

# Original function to run selected experiments

def run_selected_experiments(b):

    from IPython.display import clear_output # Moved import here for clarity
    # clear_output(wait=True) # Clear previous output first
    output_area.clear_output(wait=True) 
    with output_area:
        print("Starting experiment run...")
        # Get new directory paths from widgets
        new_data_dir = data_dir_widget.value
        new_cache_dir = cache_dir_widget.value
        
        # Define path to system_config.py (relative to notebook directory)
        # Assumes 'config' is a subdirectory of the notebook's directory
        config_file_path = os.path.join('config', 'system_config.py')
        
        try:
            print(f"Attempting to update {config_file_path}...")
            with open(config_file_path, 'r') as f:
                lines = f.readlines()
            
            new_lines = []
            config_updated = False
            for line in lines:
                if "'AUDIO_DATA_DIRECTORY':" in line:
                    # Use r-string for replacement to handle backslashes in path correctly
                    new_line = re.sub(r"('AUDIO_DATA_DIRECTORY':\s*r\")[^\"]*(\")", rf'\1{new_data_dir}\2', line)
                    if new_line != line:
                        config_updated = True
                    new_lines.append(new_line)
                elif "'CACHE_DIRECTORY':" in line: 
                    new_line = re.sub(r"('CACHE_DIRECTORY':\s*r\")[^\"]*(\")", rf'\1{new_cache_dir}\2', line)
                    if new_line != line:
                        config_updated = True
                    new_lines.append(new_line)
                else:
                    new_lines.append(line)
            
            if config_updated:
                with open(config_file_path, 'w') as f:
                    f.writelines(new_lines)
                print(f"Successfully updated {config_file_path} with new directory paths.")
            else:
                print(f"{config_file_path} already up-to-date or keys not found.")
            
            # Reload the system_config module to apply changes
            importlib.reload(config.system_config)
            # Re-import SC if it's used directly in this notebook, or ensure train_model gets the fresh one.
            # from config.system_config import SC 
            print("System configuration reloaded.")
            
        except Exception as e:
            print(f"Error updating or reloading system_config.py: {e}")
            print("Proceeding with potentially outdated configuration.")
            # Decide if you want to return or proceed if config update fails
            # return 

        selected_experiments = list(experiment_widget.value)
        if not selected_experiments:
            print("No experiment selected. Please select at least one experiment.")
            return
        
        for exp_name in selected_experiments:
            exp_config = next((exp for exp in EXPERIMENTS if exp["name"] == exp_name), None)
            if exp_config is None:
                print(f"Experiment {exp_name} not found in EXPERIMENTS.")
                continue

            print(f"Running experiment: {exp_config['name']}")
            # Pass configuration values to the train_model function.
            # train_model will use the reloaded system_config.SC for DATA_DIR and CACHE_DIR
            model, history = train_model(
                model_name=exp_config['model'],
                epochs=exp_config.get('epochs'),
                batch_size=exp_config.get('batch_size')
            )
            print(f"Training completed for experiment: {exp_config['name']}")
            if model:
                 model.summary(print_fn=lambda x: print(x)) # Ensure summary prints to output_area
            print("-" * 40)

run_selected_button.on_click(run_selected_experiments)

## 6. View Previous Results (From here down, notebook is under development)

If you've already run experiments, you can view and analyse the results here.

In [ ]:
# Under development
def load_results(output_dir=OUTPUT_DIR):
    # Check if results directory exists
    if not os.path.exists(output_dir):
        print(f"Results directory does not exist: {output_dir}")
        return None
    
    # Look for comparison report CSV
    csv_files = [f for f in os.listdir(output_dir) if f.startswith("comparison_results_") and f.endswith(".csv")]
    
    if not csv_files:
        print("No comparison results found. Run experiments or generate a report first.")
        return None
    
    # Load the latest CSV file
    latest_csv = max(csv_files)
    csv_path = os.path.join(output_dir, latest_csv)
    results_df = pd.read_csv(csv_path)
    
    print(f"Loaded results from: {csv_path}")
    return results_df

# Load and display results if available
results_df = load_results()
if results_df is not None:
    display(results_df)

## 7. Visualise Results

Create various visualisations to compare experiment results.

In [ ]:
# Under Development
# Taken from previously developed notebooks in Machine Learing course

def visualize_results(results_df):
    if results_df is None or len(results_df) == 0:
        print("No results available to visualize.")
        return
    
    # Set the figure size for better visibility
    plt.figure(figsize=(14, 8))
    
    # Create accuracy comparison bar chart
    plt.subplot(2, 2, 1)
    sns.barplot(x='Experiment', y='Test Accuracy', data=results_df)
    plt.title('Test Accuracy by Experiment')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Create F1 score comparison bar chart
    plt.subplot(2, 2, 2)
    sns.barplot(x='Experiment', y='F1 Score', data=results_df)
    plt.title('F1 Score by Experiment')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Training time comparison
    plt.subplot(2, 2, 3)
    sns.barplot(x='Experiment', y='Training Time (min)', data=results_df)
    plt.title('Training Time by Experiment (minutes)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    
    # Model comparison
    plt.subplot(2, 2, 4)
    model_comparison = results_df.groupby('Model')['Test Accuracy'].mean().reset_index()
    sns.barplot(x='Model', y='Test Accuracy', data=model_comparison)
    plt.title('Average Accuracy by Model')
    plt.tight_layout()
    
    plt.tight_layout(pad=3.0)
    plt.show()
    
    # Create a separate visualization for augmentation impact
    plt.figure(figsize=(12, 6))
       
    aug_df = pd.DataFrame(aug_data)
    sns.barplot(x='Augmentation', y='Accuracy', hue='Augmentation Type', data=aug_df)
    plt.title('Accuracy by Augmentation Type')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

# Visualize results if available
if results_df is not None:
    visualize_results(results_df)

## 8. Experiment Analysis and Conclusions
(if required)




In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import matplotlib
import librosa
import sklearn
import soundfile as sf

print("✅ TensorFlow:", tf.__version__)
print("✅ TensorFlow Hub:", hub.__version__)
print("✅ NumPy:", np.__version__)
print("✅ Pandas:", pd.__version__)
print("✅ Matplotlib:", matplotlib.__version__)
print("✅ Librosa:", librosa.__version__)
print("✅ Scikit-learn:", sklearn.__version__)
print("✅ Soundfile:", sf.__version__)


In [2]:
from config.experiment_configs import EXPERIMENTS
from config.system_config import SC
from utils.optimised_engine_pipeline import train_model
import tensorflow as tf

# 选择 baseline 实验
exp_cfg = next(e for e in EXPERIMENTS if e["name"] == "mobilenet_v3_small_baseline")

print("Data dir:", SC["AUDIO_DATA_DIRECTORY"])
print("Starting training for model:", exp_cfg["model"])

# === 训练 ===
trained_model, history = train_model(
    model_name=exp_cfg["model"],
    epochs=SC["MAX_EPOCHS"],
    batch_size=SC["BATCH_SIZE"]
)

# === 快速报告 ===
final_acc = history.history.get("accuracy", [None])[-1] if hasattr(history, "history") else None
print("Training finished.")
print("Final acc:", final_acc)


c:\Users\tianc\anaconda3\envs\project_echo_env\lib\site-packages\tensorflow_hub\__init__.py:61: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import parse_version


Data dir: C:\Users\tianc\OneDrive\桌面\Project-Echo\Project-Echo\uploads_for_training
Starting training for model: MobileNetV3-Small
Building datasets for MobileNetV3-Small...
Train dataset size (batches): 1
Validation dataset size (batches): 0
Test dataset size (batches): 1
Datasets built successfully!
Model configuration: {'hub_url': 'https://tfhub.dev/google/imagenet/mobilenet_v3_small_100_224/classification/5', 'trainable': True, 'dense_layers': [8, 4], 'dropout': 0.5, 'learning_rate': 0.0001, 'expected_input_shape': (224, 224, 3)}
Building model: MobileNetV3-Small with expected input shape: (224, 224, 3)




Starting training for model: MobileNetV3-Small


FailedPreconditionError: {{function_node __wrapped__CreateSummaryFileWriter_device_/job:localhost/replica:0/task:0/device:CPU:0}} tensorboard_logs is not a directory [Op:CreateSummaryFileWriter] name: 

In [3]:
import tensorflow as tf
import datetime

log_dir = r"C:\Users\tianc\OneDrive\桌面\Project-Echo\src\Prototypes\engine\Benchmarking_and_Experimentation\tensorboard_logs\test_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = tf.summary.create_file_writer(log_dir)

with writer.as_default():
    tf.summary.scalar("ping", 1.0, step=0)

print("日志写入成功:", log_dir)


FailedPreconditionError: {{function_node __wrapped__CreateSummaryFileWriter_device_/job:localhost/replica:0/task:0/device:CPU:0}} C:\Users\tianc\OneDrive\桌面\Project-Echo\src\Prototypes\engine\Benchmarking_and_Experimentation\tensorboard_logs is not a directory [Op:CreateSummaryFileWriter] name: 